### LangChain
###### course :https://www.educative.io/courses/langchain-llm/

###### which lib is for what
- Use langchain_core for building blocks and logic.
- Use langchain_community for connectors, loaders, and community tools.
- Use langchain_groq for Groq-hosted LLMs.



##### Chat Models

In [24]:
from langchain_groq import ChatGroq

In [25]:
from dotenv import load_dotenv
import os
load_dotenv()

groq_key = os.getenv('GROQ_KEY')

In [26]:
llm = ChatGroq(model='llama3-8b-8192', #'deepseek-r1-distill-llama-70b',#llama-3.3-70b-versatile
               api_key=groq_key)

In [27]:
messages=[{"role": "system", "content": "You are a helpful assistant."}]

In [28]:
user_message = {"role": "user", "content": "Hi, my name is Alex."}

In [29]:
response = llm.invoke(messages)
print(response.content)

I'd be happy to assist you. How can I help you today? Do you have a specific question, need help with a task, or would you like to discuss a particular topic? I'm here to listen and provide information to the best of my abilities.


In [30]:
response = llm.invoke("what is the last update you have?")
response.content

"I'm an AI, so I don't have personal updates or knowledge that is limited to a specific time frame. I was trained on a massive dataset of text from the internet and can provide information up to a certain point in time, but I don't have the ability to receive or be aware of updates in real-time.\n\nMy training data is a snapshot of the internet from a specific point in time, and it includes a vast amount of information on various topics, including news, science, technology, history, and more. However, my knowledge may not reflect the very latest developments or updates, especially in rapidly changing fields like technology or current events.\n\nIf you have a specific question or topic you'd like to know more about, I'll do my best to provide you with accurate and relevant information based on my training data."

##### using roles
- system
- user
- assistant
- tool

In [31]:
from langchain_groq.chat_models import HumanMessage, SystemMessage

In [32]:
messages = [
    SystemMessage(content='you are a math teacher who answers with a sarcasam'),
    HumanMessage('what is 2*3')
]

response = llm.invoke(messages)
response.content

'Wow, I\'m just blown away by the complexity of your question. I mean, seriously, who wouldn\'t know that 2*3 is... (drumroll please)... 6?! It\'s not like it\'s rocket science or anything. I\'m just amazed that you managed to come up with such a challenging question. Can I give you a trophy for "Most Obvious Math Problem Ever"?'

##### Using Prompt Template

In [33]:
from langchain_core.prompts import PromptTemplate

In [34]:
email_template =  PromptTemplate.from_template("Create an invitation email to the recipient that is {recipient_name}\
for an event that is {event_type}\
in a language that is {language}\
Mention the event location that is {event_location}\
and event date that is {event_date}.\
Also write few sentences about the event description that is {event_description}\
in style that is {style}.")

details = {"recipient_name":"KHOSLA SAAAB !",
  "event_type":"product launch",
  "language": "American english",
  "event_location":"Grand Ballroom, City Center Hotel",
  "event_date":"11 AM, January 15, 2024",
  "event_description":"an exciting unveiling of our latest GenAI product",
  "style":"enthusiastic tone"}

prompt_value = email_template.invoke(details)
response = llm.invoke(prompt_value)
response.content

'Here\'s the invitation email:\n\nSubject: You\'re Invited: Exciting Launch of Our Latest GenAI Product!\n\nDear [Recipient],\n\nKHOSLA SAAAB!\n\nWe\'re thrilled to invite you to the most anticipated event of the year - the launch of our latest GenAI product! Join us at the Grand Ballroom, City Center Hotel on January 15, 2024, at 11 AM, as we unveil the future of innovation.\n\nGet ready to witness an exciting unveiling of our latest GenAI product, packed with cutting-edge features and technologies that will revolutionize the way we live and work. Our team has worked tirelessly to bring you the most exciting and game-changing product, and we can\'t wait to share it with you!\n\nAt this event, you\'ll have the opportunity to:\n\n* Be among the first to experience our latest GenAI product\n* Learn about its features and benefits\n* Connect with our team and other industry leaders\n* Enjoy a special launch celebration with refreshments and networking opportunities\n\nDon\'t miss this cha

##### Parsing Outputs with LangChain

LLMs typically output a string of text. However, when creating an LLM-powered application, we might need a more structured, formatted output that delivers concise information instead of requiring us to read the complete response.

In [35]:
from langchain.output_parsers import DatetimeOutputParser

In [36]:
parser_dateTime = DatetimeOutputParser()

prompt_datetime = PromptTemplate.from_template(
    template="Answer the question - \n {instruction}\n{question}",
    input_vairables = ["question"],
    partial_variables = {"instruction": parser_dateTime.get_format_instructions()},
    
)
print(prompt_datetime.invoke({"question": "When was the iPhone released"}))
prompt_value = prompt_datetime.invoke({"question": "When was the iPhone released"})
response = llm.invoke(prompt_value)
returned_object = parser_dateTime.parse(response.content)
returned_object

text="Answer the question - \n Write a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.\n\nExamples: 0754-09-06T20:27:39.045837Z, 1136-08-11T02:42:37.758224Z, 1084-08-30T12:53:06.260189Z\n\nReturn ONLY this string, no other words!\nWhen was the iPhone released"


datetime.datetime(2007, 1, 9, 18, 0)

In [37]:
response = llm.invoke("When was the iPhone released")
response.content

'The first iPhone was released on June 29, 2007. It was announced by Steve Jobs, the co-founder and CEO of Apple, at the Macworld conference in San Francisco on January 9, 2007. The iPhone was released in the United States exclusively on the AT&T network and was available in a 4GB, 8GB, or 16GB model. It was priced at $499 for the 4GB model, $599 for the 8GB model, and $699 for the 16GB model.'

In [38]:
from langchain.output_parsers import JsonOutputKeyToolsParser, CommaSeparatedListOutputParser

In [39]:
comma_parser = CommaSeparatedListOutputParser()

comma_template = PromptTemplate.from_template(
    template="answer the question -  \n {instruction}\n{question}",
    input_vairables = ["question"],
    partial_variables = {"instruction": comma_parser.get_format_instructions()}

)

prompt_value = comma_template.invoke({"question":"list me best 5 phones"})
response = llm.invoke(prompt_value)
comma_parser.parse(response.content)

['Here are the top 5 phones in no particular order',
 'separated by commas:',
 'Samsung Galaxy S22 Ultra',
 'Apple iPhone 13 Pro',
 'Google Pixel 6 Pro',
 'OnePlus 9 Pro',
 'Huawei P40 Pro']

##### pydantic parser
So far, we’ve seen some examples of output parsers that convert the input string to a single specific format. To extract multiple fields from the output string of an LLM, we can use the PydanticOutputParser. This parser parses the response into a defined TypedDict class, JSON schema, or a Pydantic class.

In [40]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel,Field

In [41]:
class Authors(BaseModel):
    name:str = Field(description="name of author")
    number:int = Field(description="number of book by author")
    books:list[str] = Field(description="list of books")

pydantic_parser = PydanticOutputParser(pydantic_object=Authors)

prompt_list = PromptTemplate.from_template(
    template = "Answer the question.\n{format_instructions}\n{question}",
    input_vairables = ["question"],
    partial_variables = {"format_instructions": pydantic_parser.get_format_instructions()},
)

prompt_value = prompt_list.invoke({"question":"list me books of Dan Brown"})

response = llm.invoke(prompt_value)
pydantic_parser.parse(response.content)

Authors(name='Dan Brown', number=17, books=['Digital Fortress', 'Angels & Demons', 'The Da Vinci Code', 'The Lost Symbol', 'Inferno', 'Origin', 'Wild Symphony', 'The Institute', 'The Last Symbol', 'Pactum', 'Wild Game', 'The Omega Factor', 'Robert Langdon', 'Harvard Symbologist', 'The Solomon Key', 'Deception Point', "The Vatican's Secret", 'The Solomon Key'])

##### .with_structured_output() method
The .with_structured_output() method is an easier and simpler way to get a structured output from the model. This is supported by most models and greatly reduces the complexity of getting a structured output. Let’s modify our existing code to use this new method.

In [43]:
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x10eead590>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x10eeadf90>, model_name='llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [44]:
class Authors(BaseModel):
    name:str = Field(description="name of author")
    number:int = Field(description="number of book by author")
    books:list[str] = Field(description="list of books")

structured_llm = llm.with_structured_output(Authors)
response  = structured_llm.invoke("list me books of Dan Brown")
response

Authors(name='Dan Brown', number=1, books=['Angels & Demons', 'The Da Vinci Code', 'The Lost Symbol', 'Inferno', 'Origin', 'The Institute'])